In [1]:
import requests
import json
import pandas as pd

from confluent_kafka import Producer
from confluent_kafka.admin import AdminClient, NewTopic

In [4]:
# Get User data from dummyAPI
resp = requests.get('https://dummyjson.com/users?limit=5')
user_dict = resp.json()

print(type(user_dict))

<class 'dict'>


In [5]:
# Using json.dumps to print data inside JSON
user_json = json.dumps(user_dict, indent=4)
print(user_json)

{
    "users": [
        {
            "id": 1,
            "firstName": "Terry",
            "lastName": "Medhurst",
            "maidenName": "Smitham",
            "age": 50,
            "gender": "male",
            "email": "atuny0@sohu.com",
            "phone": "+63 791 675 8914",
            "username": "atuny0",
            "password": "9uQFF1Lh",
            "birthDate": "2000-12-25",
            "image": "https://robohash.org/hicveldicta.png",
            "bloodGroup": "A\u2212",
            "height": 189,
            "weight": 75.4,
            "eyeColor": "Green",
            "hair": {
                "color": "Black",
                "type": "Strands"
            },
            "domain": "slashdot.org",
            "ip": "117.29.86.254",
            "address": {
                "address": "1745 T Street Southeast",
                "city": "Washington",
                "coordinates": {
                    "lat": 38.867033,
                    "lng": -76.979235
            

In [6]:
# Convert JSON data to Pandas dataframe to see it mroe clearly
df = pd.DataFrame(user_dict.get('users'))
df.head(2)

,id,firstName,lastName,maidenName,age,gender,email,phone,username,password,...,domain,ip,address,macAddress,university,bank,company,ein,ssn,userAgent
0,1,Terry,Medhurst,Smitham,50,male,atuny0@sohu.com,+63 791 675 8914,atuny0,9uQFF1Lh,...,slashdot.org,117.29.86.254,"{'address': '1745 T Street Southeast', 'city':...",13:69:BA:56:A3:74,Capitol University,"{'cardExpire': '06/22', 'cardNumber': '5038095...","{'address': {'address': '629 Debbie Drive', 'c...",20-9487066,661-64-2976,Mozilla/5.0 (Windows NT 6.1) AppleWebKit/534.2...
1,2,Sheldon,Quigley,Cole,28,male,hbingley1@plala.or.jp,+7 813 117 7139,hbingley1,CQutx25i8r,...,51.la,253.240.20.181,"{'address': '6007 Applegate Lane', 'city': 'Lo...",13:F1:00:DA:A4:12,Stavropol State Technical University,"{'cardExpire': '10/23', 'cardNumber': '5355920...",{'address': {'address': '8821 West Myrtle Aven...,52-5262907,447-08-9217,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...


In [7]:
# Create an AdminClient
ac = AdminClient({'bootstrap.servers': 'localhost'})

In [8]:
# Create a topic
topic = NewTopic('user', num_partitions=2, replication_factor=1)

In [9]:
topic

NewTopic(topic=user,num_partitions=2)

In [10]:
product_topic = ac.create_topics([topic])

In [11]:
product_topic.values()

dict_values([<Future at 0x16e04c9da20 state=finished returned NoneType>])

In [12]:
#Create Producer
producer = Producer({'bootstrap.servers': 'localhost'})

In [13]:
user_data = user_dict["users"]
for data in user_data:
    print(data)

{'id': 1, 'firstName': 'Terry', 'lastName': 'Medhurst', 'maidenName': 'Smitham', 'age': 50, 'gender': 'male', 'email': 'atuny0@sohu.com', 'phone': '+63 791 675 8914', 'username': 'atuny0', 'password': '9uQFF1Lh', 'birthDate': '2000-12-25', 'image': 'https://robohash.org/hicveldicta.png', 'bloodGroup': 'A−', 'height': 189, 'weight': 75.4, 'eyeColor': 'Green', 'hair': {'color': 'Black', 'type': 'Strands'}, 'domain': 'slashdot.org', 'ip': '117.29.86.254', 'address': {'address': '1745 T Street Southeast', 'city': 'Washington', 'coordinates': {'lat': 38.867033, 'lng': -76.979235}, 'postalCode': '20020', 'state': 'DC'}, 'macAddress': '13:69:BA:56:A3:74', 'university': 'Capitol University', 'bank': {'cardExpire': '06/22', 'cardNumber': '50380955204220685', 'cardType': 'maestro', 'currency': 'Peso', 'iban': 'NO17 0695 2754 967'}, 'company': {'address': {'address': '629 Debbie Drive', 'city': 'Nashville', 'coordinates': {'lat': 36.208114, 'lng': -86.58621199999999}, 'postalCode': '37076', 'stat

In [18]:
for val in user_data:
    send_msg = str(val)

    # send data using the producer

    producer.produce('user', send_msg.encode('utf-8'),
        callback = lambda err,msg : print("Message Delivered to Topic => ", msg.topic(),
                                            " @Partition => ", msg.partition()) )

    #
    producer.flush()

Message Delivered to Topic =>  user  @Partition =>  0
Message Delivered to Topic =>  user  @Partition =>  0
Message Delivered to Topic =>  user  @Partition =>  0
Message Delivered to Topic =>  user  @Partition =>  0
Message Delivered to Topic =>  user  @Partition =>  0
